In [1]:
import pandas as pd         #引入pandas套件
import numpy as np

#usecols = ['主要用途','建物型態','總樓層數','交易筆棟數','總價元','車位總價元']     #選取需要的檔案及其欄位匯入
df_a = pd.read_csv('/Users/andywei/Desktop/download/a_lvr_land_a.csv' , encoding = "utf_8_sig")   #匯入指定地區的.csv檔案
df_a = df_a.drop(index=0)                                                                         #刪除英文表頭，英文表頭所在的index為0
df_b = pd.read_csv('/Users/andywei/Desktop/download/b_lvr_land_a.csv' , encoding = "utf_8_sig")
df_b = df_b.drop(index=0)
df_e = pd.read_csv('/Users/andywei/Desktop/download/e_lvr_land_a.csv' , encoding = "utf_8_sig")
df_e = df_e.drop(index=0)
df_f = pd.read_csv('/Users/andywei/Desktop/download/f_lvr_land_a.csv' , encoding = "utf_8_sig")
df_f = df_f.drop(index=0)
df_h = pd.read_csv('/Users/andywei/Desktop/download/h_lvr_land_a.csv' , encoding = "utf_8_sig")
df_h = df_h.drop(index=0)

df_all = pd.concat([df_a, df_b, df_e, df_f, df_h])  #對dataframes進行合併
df_all = df_all.reset_index(drop = True)  #合併後重新整理index


#建立一個國字轉數字的function
def c2n(a):
    if a == '一':
        return 1
    elif a == '二':
        return 2
    elif a == '三':
        return 3
    elif a == '四':
        return 4
    elif a == '五':
        return 5
    elif a == '六':
        return 6
    elif a == '七':
        return 7
    elif a == '八':
        return 8
    elif a == '九':
        return 9

#建立一個將中文數字句轉換為數字的function
def floor2num(d, l, i): #d:dataframe; l:label; i:index

    chl = list(d[l][i])  #將餵入的string轉成list
        
    if len(chl) == 4:
        return c2n(chl[0])*10 + c2n(chl[2])
    elif len(chl) == 3:
        if chl[0] == '十':
            return c2n(chl[1]) + 10
        elif chl[1] =='十':
            return c2n(chl[0]) * 10
    elif len(chl) == 2:
        return c2n(chl[0])
    else:
        print('error when translate chinese num to num')
        return np.nan

#建立一個用來存放數字總樓層數的col
df_all = df_all.assign(數字總樓層數 = '')#筆記：keyword不能用引號匡起來

#以for回圈逐筆將國字的總樓層數轉為數字，並存放在新建立的'數字總樓層數'欄位中。
for i in df_all.index:
    try:
        df_all['數字總樓層數'][i] = int(floor2num(df_all, '總樓層數', i))
    except:
        #用來catch‘總樓層數’為空值的row並回填空值以略過此筆資料
        df_all['數字總樓層數'][i] = np.nan
        
        
#以上前處理完畢。

#filter_a條件設定
filter_a_con = (df_all.數字總樓層數 >= 13) & (df_all.主要用途 == '住家用') & (df_all.建物型態 =='住宅大樓(11層含以上有電梯)') 
df_filter_a = df_all[filter_a_con]
print(df_filter_a)

#將套用條件後的dataframe轉存為.csv檔案
df_filter_a.to_csv('filter_a.csv', encoding = 'utf_8_sig')



     鄉鎮市區          交易標的                土地位置建物門牌 土地移轉總面積平方公尺  \
5     文山區  房地(土地+建物)+車位  臺北市文山區興隆路四段１４５巷２７號五樓之２        16.4   
7     中正區  房地(土地+建物)+車位         臺北市中正區牯嶺街１４６號六樓       19.79   
16    中山區  房地(土地+建物)+車位      臺北市中山區南京東路三段１０６號七樓       15.88   
35    中山區  房地(土地+建物)+車位      臺北市中山區南京東路三段１０８號八樓       16.06   
36    內湖區  房地(土地+建物)+車位   臺北市內湖區內湖路二段１０３巷１６號十五樓      100.79   
...   ...           ...                     ...         ...   
7171  桃園區  房地(土地+建物)+車位       桃園市桃園區新埔八街１２０號十四樓        24.2   
7172  桃園區  房地(土地+建物)+車位        桃園市桃園區新埔八街１２６號五樓       26.03   
7173  桃園區  房地(土地+建物)+車位        桃園市桃園區新埔八街１２６號七樓       26.03   
7174  桃園區  房地(土地+建物)+車位       桃園市桃園區新埔八街１３０號十一樓       20.21   
7175  桃園區  房地(土地+建物)+車位       桃園市桃園區新埔八街１１６號十一樓       24.89   

                                               都市土地使用分區 非都市土地使用分區 非都市土地使用編定  \
5                                         都市：其他:第三種住宅區。       NaN       NaN   
7                                         都市：其他:第三種住宅區。       NaN       NaN   
16    

In [84]:
import qgrid                #引入qgrid套件以便視覺化操作

#print(df_filter_a)
df_filter_a.to_csv("filter_a.csv", encoding = 'utf_8_sig')
qq = qgrid.show_grid(df_filter_a)
qq

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [147]:
#建立一個剖析'交易筆棟數'並回傳取得的車位數量的function，
def calcps(d, i): #d: dataframe; i: index
    transacnum = list(d['交易筆棟數'][i]) #將'交易筆棟數'的內容放入一list
    #print(transacnum)
    ps = ''  #ParkingSpace
    while(1):
        #筆記：pop()：取出list的最後一項元素。
        psb = transacnum.pop() #自list末端pop出最後一項元素並放入parkingspacebuffer暫存。
        if psb != '位': #如果pop出的元素不是'位'字，則繼續pop。這麼做是為使>10個車位的資料也能正確取出。
            ps = str(psb) + str(ps) #新取出來的元素，會由ps字串的前面插入
        else:
            break #找到'位'字後跳出while true.
    return int(ps) #將字串轉為整數後回傳.
#############################################################################################

#總件數
totalitems = len(df_all.index)
#print('總件數:' + str(totalitems))

#總車位數
total_ps = 0
for i in df_all.index:
    total_ps += calcps(df_all, i)
    
#print('總車位數:' + str(total_ps))

#平均總價元
total_price = 0
for i in df_all.index:
    total_price += int(df_all['總價元'][i])
avg_total_price = total_price / totalitems

#print('平均總價元:' + str(round(avg_total_price, 2)))

#平均車位總價元
total_ps_price = 0
for i in df_all.index:
    total_ps_price += int(df_all['車位總價元'][i])
avg_ps_price = total_ps_price / total_ps

#print('平均車位總價元:' + str(round(avg_ps_price,2)))

df_filter_b = pd.DataFrame({'總件數': [totalitems],
                           '總車位數': [total_ps],
                            '平均總價元': [round(avg_total_price, 2)],
                            '平均車位總價元': [round(avg_ps_price,2)]
                           })
print(df_filter_b)
df_filter_b.to_csv('filter_b.csv', encoding = 'utf_8_sig')

    總件數  總車位數        平均總價元    平均車位總價元
0  7176  3784  15588346.57  922056.42
